In [1]:
import numpy as np
import pandas as pd

In [3]:
Reviews = pd.read_csv("C:\\Users\\TRSM\\Downloads\\Telegram Desktop\\Review.csv")

In [4]:
Reviews

,User ID,Product ID,Date,Review,Rating,Label
0,923,0,2014-12-08,The food at snack is a selection of popular Gr...,3,-1
1,924,0,2013-05-16,This little place in Soho is wonderful. I had ...,3,-1
2,925,0,2013-07-01,ordered lunch for 15 from Snack last Friday. ...,4,-1
3,926,0,2011-07-28,This is a beautiful quaint little restaurant o...,4,-1
4,927,0,2010-11-01,Snack is great place for a casual sit down lu...,4,-1
...,...,...,...,...,...,...
358952,161146,349,2014-02-06,"I'm very spoiled with Pizza. Really, I have tr...",2,1
358953,116424,349,2014-01-31,Can't say enough good things about this place....,5,1
358954,161147,349,2014-01-30,"Had a great dinner here- fantastic pizza, the ...",2,1
358955,97930,349,2014-01-25,"Great foods and great drinks, they have even p...",5,1


In [5]:
missing_counts = Reviews.isnull().sum()

In [6]:
missing_counts

User ID       0
Product ID    0
Date          0
Review        0
Rating        0
Label         0
dtype: int64

In [7]:
Reviews['Date'] = pd.to_datetime(Reviews['Date'])
# Replace the date column with numerical value
Reviews['Date'] = (Reviews['Date'] - Reviews['Date'].min()).dt.days

In [8]:
Reviews

,User ID,Product ID,Date,Review,Rating,Label
0,923,0,3701,The food at snack is a selection of popular Gr...,3,-1
1,924,0,3130,This little place in Soho is wonderful. I had ...,3,-1
2,925,0,3176,ordered lunch for 15 from Snack last Friday. ...,4,-1
3,926,0,2472,This is a beautiful quaint little restaurant o...,4,-1
4,927,0,2203,Snack is great place for a casual sit down lu...,4,-1
...,...,...,...,...,...,...
358952,161146,349,3396,"I'm very spoiled with Pizza. Really, I have tr...",2,1
358953,116424,349,3390,Can't say enough good things about this place....,5,1
358954,161147,349,3389,"Had a great dinner here- fantastic pizza, the ...",2,1
358955,97930,349,3384,"Great foods and great drinks, they have even p...",5,1


In [9]:
#finding product IDs with many real or fake reveiws as outlier to remove them
review_counts = Reviews.groupby(['Product ID', 'Label']).size().unstack(fill_value=0)
review_counts = review_counts.reset_index()
review_counts.columns.name = None
review_counts.rename(columns={'fake': 'Fake Reviews', 'genuine': 'Genuine Reviews'}, inplace=True)


In [10]:
review_counts

,Product ID,-1,1
0,0,14,196
1,1,42,211
2,2,19,147
3,3,9,50
4,4,37,510
...,...,...,...
918,918,21,368
919,919,82,404
920,920,28,278
921,921,33,159


In [11]:
#now by Z-score statistical method I want to specify the product ID outliers
from scipy import stats
review_counts['Fake Z-score'] = np.abs(stats.zscore(review_counts[-1]))
review_counts['Real Z-score'] = np.abs(stats.zscore(review_counts[1]))
z_threshold = 3
fake_outliers = review_counts[review_counts['Fake Z-score'] > z_threshold]
real_outliers = review_counts[review_counts['Real Z-score'] > z_threshold]
print("Outliers in the 'Fake Reviews' column:")
print(fake_outliers)

print("\nOutliers in the 'Real Reviews' column:")
print(real_outliers)


Outliers in the 'Fake Reviews' column:
     Product ID   -1     1  Fake Z-score  Real Z-score
78           78  249  1431      3.200045      2.122329
120         120  411  1562      5.680013      2.379270
211         211  271  2577      3.536831      4.370069
247         247  704  6674     10.165387     12.405839
363         363  316  1749      4.225711      2.746047
416         416  353  1830      4.792123      2.904919
454         454  249  1909      3.200045      3.059868
465         465  544  3391      7.716036      5.966632
484         484  247  1685      3.169428      2.620519
521         521  409  1663      5.649396      2.577369
542         542  701  4015     10.119462      7.190532
555         555  797  5835     11.589072     10.760242
658         658  444  1721      6.185192      2.691129
668         668  362  2581      4.929899      4.377915

Outliers in the 'Real Reviews' column:
     Product ID   -1     1  Fake Z-score  Real Z-score
72           72  194  2805      2.358081 

In [12]:
outlier_product_ids = list(set(fake_outliers['Product ID']).union(set(real_outliers['Product ID'])))


In [13]:
outlier_product_ids

[454,
 72,
 521,
 78,
 465,
 658,
 211,
 468,
 91,
 668,
 604,
 542,
 414,
 416,
 484,
 100,
 363,
 555,
 247,
 120,
 510]

In [14]:
filtered_reviews = Reviews[~Reviews['Product ID'].isin(outlier_product_ids)]


In [15]:
filtered_reviews

,User ID,Product ID,Date,Review,Rating,Label
0,923,0,3701,The food at snack is a selection of popular Gr...,3,-1
1,924,0,3130,This little place in Soho is wonderful. I had ...,3,-1
2,925,0,3176,ordered lunch for 15 from Snack last Friday. ...,4,-1
3,926,0,2472,This is a beautiful quaint little restaurant o...,4,-1
4,927,0,2203,Snack is great place for a casual sit down lu...,4,-1
...,...,...,...,...,...,...
358952,161146,349,3396,"I'm very spoiled with Pizza. Really, I have tr...",2,1
358953,116424,349,3390,Can't say enough good things about this place....,5,1
358954,161147,349,3389,"Had a great dinner here- fantastic pizza, the ...",2,1
358955,97930,349,3384,"Great foods and great drinks, they have even p...",5,1


In [33]:
#now seeing the User ID distribution
user_review_counts = Reviews.groupby(['User ID', 'Label']).size().unstack(fill_value=0)
user_review_counts = user_review_counts.reset_index()
user_review_counts.columns.name = None
user_review_counts.rename(columns={-1: 'Fake Reviews', 1: 'Genuine Reviews'}, inplace=True)


In [34]:
user_review_counts

,User ID,Fake Reviews,Genuine Reviews
0,923,13,26
1,924,1,0
2,925,2,0
3,926,1,0
4,927,2,3
...,...,...,...
160196,161143,0,1
160197,161144,0,1
160198,161145,0,1
160199,161146,0,1


In [37]:
#now by Z-score statistical method I want to specify the User ID outliers however I didn't run this for the last version
from scipy import stats
user_review_counts['Fake Z-score'] = np.abs(stats.zscore(user_review_counts["Fake Reviews"]))
user_review_counts['Real Z-score'] = np.abs(stats.zscore(user_review_counts["Genuine Reviews"]))
z_threshold = 3
fake_user_outliers = user_review_counts[user_review_counts['Fake Z-score'] > z_threshold]
real_user_outliers = user_review_counts[user_review_counts['Real Z-score'] > z_threshold]
print("Outliers in the 'Fake Reviews' column:")
print(fake_user_outliers)

print("\nOutliers in the 'Real Reviews' column:")
print(real_user_outliers)

Outliers in the 'Fake Reviews' column:
        User ID  Fake Reviews  Genuine Reviews  Fake Z-score  Real Z-score
0           923            13               26     21.170867      6.081412
8           931            20               11     32.776105      2.278878
39          962             6               67      9.565630     16.475006
40          963             3               35      4.591957      8.362933
43          966             3               60      4.591957     14.700490
...         ...           ...              ...           ...           ...
120777   121723             3                0      4.591957      0.509647
120967   121913             3                0      4.591957      0.509647
122045   122991             3                3      4.591957      0.250860
124648   125594             3                0      4.591957      0.509647
135670   136617             3                3      4.591957      0.250860

[1404 rows x 5 columns]

Outliers in the 'Real Reviews' colu

In [38]:
outlier_user_ids = list(set(fake_user_outliers['User ID']).union(set(real_user_outliers['User ID'])))

In [39]:
outlier_user_ids

[16385,
 8196,
 8200,
 8201,
 8203,
 8206,
 8207,
 16407,
 16421,
 8229,
 8232,
 16428,
 8240,
 16433,
 32821,
 16438,
 8251,
 8257,
 32838,
 24651,
 57420,
 49233,
 8274,
 8275,
 24662,
 8279,
 8280,
 8281,
 32855,
 8284,
 57437,
 41054,
 8289,
 49254,
 8296,
 57449,
 122991,
 8305,
 32886,
 16504,
 32890,
 8320,
 16513,
 49281,
 8325,
 8326,
 16519,
 24715,
 8332,
 8336,
 8337,
 8339,
 8352,
 16545,
 8355,
 24740,
 16551,
 16552,
 8359,
 57514,
 8361,
 41133,
 8366,
 24754,
 16563,
 16570,
 24769,
 16578,
 8387,
 8388,
 24771,
 24773,
 8397,
 57575,
 8436,
 8440,
 8442,
 16635,
 16636,
 16637,
 16642,
 8454,
 8455,
 16650,
 16652,
 24845,
 8460,
 24847,
 8472,
 8476,
 74013,
 16672,
 74028,
 82237,
 49469,
 24908,
 41299,
 8535,
 8536,
 33115,
 8542,
 8543,
 16738,
 16740,
 74086,
 33127,
 8553,
 8554,
 8558,
 16750,
 49520,
 8560,
 33135,
 8571,
 16764,
 33152,
 49547,
 24972,
 8593,
 16793,
 16800,
 25001,
 8621,
 16816,
 82357,
 16825,
 8634,
 8636,
 8638,
 33223,
 16840,
 8657,
 

In [41]:
filtered_reviews2 = filtered_reviews[~filtered_reviews['User ID'].isin(outlier_user_ids)]

In [42]:
filtered_reviews2

,User ID,Product ID,Date,Review,Rating,Label
1,924,0,3130,This little place in Soho is wonderful. I had ...,3,-1
2,925,0,3176,ordered lunch for 15 from Snack last Friday. ...,4,-1
3,926,0,2472,This is a beautiful quaint little restaurant o...,4,-1
4,927,0,2203,Snack is great place for a casual sit down lu...,4,-1
5,928,0,1778,A solid 4 stars for this greek food spot. If ...,4,-1
...,...,...,...,...,...,...
358950,14671,349,3398,Made a reservation for an early dinner Saturda...,3,1
358952,161146,349,3396,"I'm very spoiled with Pizza. Really, I have tr...",2,1
358953,116424,349,3390,Can't say enough good things about this place....,5,1
358954,161147,349,3389,"Had a great dinner here- fantastic pizza, the ...",2,1


In [16]:
FinalRev = filtered_reviews

In [17]:
FinalRev

,User ID,Product ID,Date,Review,Rating,Label
0,923,0,3701,The food at snack is a selection of popular Gr...,3,-1
1,924,0,3130,This little place in Soho is wonderful. I had ...,3,-1
2,925,0,3176,ordered lunch for 15 from Snack last Friday. ...,4,-1
3,926,0,2472,This is a beautiful quaint little restaurant o...,4,-1
4,927,0,2203,Snack is great place for a casual sit down lu...,4,-1
...,...,...,...,...,...,...
358952,161146,349,3396,"I'm very spoiled with Pizza. Really, I have tr...",2,1
358953,116424,349,3390,Can't say enough good things about this place....,5,1
358954,161147,349,3389,"Had a great dinner here- fantastic pizza, the ...",2,1
358955,97930,349,3384,"Great foods and great drinks, they have even p...",5,1


In [18]:
import nltk
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize
from nltk.stem import WordNetLemmatizer
import re


nltk.download('punkt')
nltk.download('stopwords')
nltk.download('wordnet')
nltk.download('omw-1.4')

def preprocess_text(text):
    # Convert to lowercase
    text = text.lower()

    # Tokenize the text
    tokens = word_tokenize(text)

    # Remove stop words and numbers
    stop_words = set(stopwords.words('english'))
    tokens = [token for token in tokens if token not in stop_words and not token.isdigit()]

    # Lemmatization
    lemmatizer = WordNetLemmatizer()
    tokens = [lemmatizer.lemmatize(token) for token in tokens]

    # Extract punctuation marks as new feature
    punctuation = [token for token in tokens if not token.isalnum()]

    # Remove punctuation tokens from the original tokens
    tokens = [token for token in tokens if token.isalnum()]

    # Join the tokens and punctuation back into separate strings
    tokens_str = ' '.join(tokens)
    punctuation_str = ' '.join(punctuation)

    return tokens_str, punctuation_str


FinalRev[['Tokens', 'Punctuation']] = FinalRev['Review'].apply(preprocess_text).apply(pd.Series)


[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\TRSM\AppData\Roaming\nltk_data...
[nltk_data]   Unzipping tokenizers\punkt.zip.
[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\TRSM\AppData\Roaming\nltk_data...
[nltk_data]   Unzipping corpora\stopwords.zip.
[nltk_data] Downloading package wordnet to
[nltk_data]     C:\Users\TRSM\AppData\Roaming\nltk_data...
[nltk_data] Downloading package omw-1.4 to
[nltk_data]     C:\Users\TRSM\AppData\Roaming\nltk_data...
C:\Users\TRSM\AppData\Local\Temp\ipykernel_6088\2562982414.py:41: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  FinalRev[['Tokens', 'Punctuation']] = FinalRev['Review'].apply(preprocess_text).apply(pd.Series)
C:\Users\TRSM\AppData\Local\Temp\ipykernel_608

In [19]:
FinalRev


,User ID,Product ID,Date,Review,Rating,Label,Tokens,Punctuation
0,923,0,3701,The food at snack is a selection of popular Gr...,3,-1,food snack selection popular greek dish appeti...,. . . 4-5 's .
1,924,0,3130,This little place in Soho is wonderful. I had ...,3,-1,little place soho wonderful lamb sandwich glas...,". . , , . , , n't world-class ."
2,925,0,3176,ordered lunch for 15 from Snack last Friday. ...,4,-1,ordered lunch snack last friday time nothing m...,". , . , ."
3,926,0,2472,This is a beautiful quaint little restaurant o...,4,-1,beautiful quaint little restaurant pretty stre...,". 're , . , . 'm . - , - . . ."
4,927,0,2203,Snack is great place for a casual sit down lu...,4,-1,snack great place casual sit especially cold w...,"lunch- . food- . . ... . . n't , ... `` '' ."
...,...,...,...,...,...,...,...,...
358952,161146,349,3396,"I'm very spoiled with Pizza. Really, I have tr...",2,1,spoiled pizza really tried lucali di fara john...,"'m . , 's , 's , 's , . , 'm . 's , . . . ! , ..."
358953,116424,349,3390,Can't say enough good things about this place....,5,1,ca say enough good thing place stopped last ni...,"n't . , pre-dinner . , ; , , . . , , , . , ."
358954,161147,349,3389,"Had a great dinner here- fantastic pizza, the ...",2,1,great dinner fantastic pizza pizza dessert bit...,"here- , s'more . . , ."
358955,97930,349,3384,"Great foods and great drinks, they have even p...",5,1,great food great drink even pairing wine pizza...,", ! ! . , , , : )"


In [20]:

from IPython.display import HTML

# Convert DataFrame to Excel and display as link
filename = 'FinalRev.xlsx'
FinalRev.to_excel(filename, index=False, sheet_name='Sheet1')
HTML(f'<a href="{filename}" download>Download Excel file</a>')


In [1]:
import pandas as pd
Nowrev = pd.read_csv('FinalRev.csv')

In [3]:
!pip install pandas


     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 12.4/12.4 MB 12.7 MB/s eta 0:00:00m eta 0:00:010:00:01
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 502.3/502.3 KB 1.7 MB/s eta 0:00:00m eta 0:00:01
  Using cached numpy-1.24.2-cp39-cp39-manylinux_2_17_x86_64.manylinux2014_x86_64.whl (17.3 MB)
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 341.8/341.8 KB 741.7 kB/s eta 0:00:00m eta 0:00:01
You should consider upgrading via the '/home/seankhatiri/sean2/venv/bin/python -m pip install --upgrade pip' command.


In [2]:
print(Nowrev.isnull().sum())

User ID           0
Product ID        0
Date              0
Review            4
Rating            0
Label             0
Tokens           64
Punctuation    1671
dtype: int64


In [3]:
Nowrev.dropna(subset=['Tokens'], inplace=True)

In [8]:
!pip install scikit-learn


  Using cached scikit_learn-1.2.2-cp39-cp39-manylinux_2_17_x86_64.manylinux2014_x86_64.whl (9.6 MB)
  Using cached joblib-1.2.0-py3-none-any.whl (297 kB)
  Using cached threadpoolctl-3.1.0-py3-none-any.whl (14 kB)
  Using cached scipy-1.10.1-cp39-cp39-manylinux_2_17_x86_64.manylinux2014_x86_64.whl (34.5 MB)
You should consider upgrading via the '/home/seankhatiri/sean2/venv/bin/python -m pip install --upgrade pip' command.


In [4]:
from sklearn.model_selection import train_test_split

#split the data into training and testing sets
X = Nowrev[['Date', 'Tokens', 'Rating']]
y = Nowrev['Label']
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)


In [4]:
#from here is BERT however I couldn't run it finally
#pip install torch torchvision transformers


  Using cached torch-2.0.0-cp39-cp39-manylinux1_x86_64.whl (619.9 MB)
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 6.0/6.0 MB 10.0 MB/s eta 0:00:00m eta 0:00:010:00:01
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 6.8/6.8 MB 6.8 MB/s eta 0:00:000m eta 0:00:010:00:01
  Using cached nvidia_cublas_cu11-11.10.3.66-py3-none-manylinux1_x86_64.whl (317.1 MB)
  Using cached nvidia_cuda_nvrtc_cu11-11.7.99-2-py3-none-manylinux1_x86_64.whl (21.0 MB)
  Using cached nvidia_nvtx_cu11-11.7.91-py3-none-manylinux1_x86_64.whl (98 kB)
  Using cached triton-2.0.0-1-cp39-cp39-manylinux2014_x86_64.manylinux_2_17_x86_64.whl (63.3 MB)
  Using cached nvidia_cufft_cu11-10.9.0.58-py3-none-manylinux1_x86_64.whl (168.4 MB)
  Using cached nvidia_cudnn_cu11-8.5.0.96-2-py3-none-manylinux1_x86_64.whl (557.1 MB)
  Using cached nvidia_cusolver_cu11-11.4.0.1-2-py3-none-manylinux1_x86_64.whl (102.6 MB)
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.1/2.1 MB 13.0 MB/s eta 0:00:00m eta 0:00:01
  Using cached nvidia_nccl_c

In [5]:
#import torch
#from torch.utils.data import DataLoader, Dataset, RandomSampler, SequentialSampler
#from transformers import BertForSequenceClassification, BertTokenizer, BertConfig, AdamW, get_linear_schedule_with_warmup


/home/seankhatiri/sean2/venv/lib/python3.9/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [27]:

#reviews = Nowrev['Tokens'].values
#labels = Nowrev['Label'].values



In [28]:
#labels = (labels + 1)//2
#labels

array([0, 0, 0, ..., 1, 1, 1])

In [29]:
# class FakeReviewDataset(Dataset):
#     def __init__(self, reviews, labels, tokenizer, max_length):
#         self.reviews = reviews
#         self.labels = labels
#         self.tokenizer = tokenizer
#         self.max_length = max_length

#     def __len__(self):
#         return len(self.reviews)

#     def __getitem__(self, idx):
#         review = self.reviews[idx]
#         label = self.labels[idx]

#         encoding = self.tokenizer.encode_plus(
#             review,
#             add_special_tokens=True,
#             max_length=self.max_length,
#             return_token_type_ids=False,
#             padding='max_length',
#             return_attention_mask=True,
#             return_tensors='pt',
#             truncation=True
#         )

#         return {
#             'review_text': review,
#             'input_ids': encoding['input_ids'].flatten(),
#             'attention_mask': encoding['attention_mask'].flatten(),
#             'label': torch.tensor(label, dtype=torch.long)
#         }

# tokenizer = BertTokenizer.from_pretrained('bert-base-uncased')
# max_length = 128 

In [30]:
# train_reviews, val_reviews, train_labels, val_labels = train_test_split(reviews, labels, test_size=0.2, random_state=42)

# train_dataset = FakeReviewDataset(train_reviews, train_labels, tokenizer, max_length)
# val_dataset = FakeReviewDataset(val_reviews, val_labels, tokenizer, max_length)


In [31]:
# batch_size = 8

# train_dataloader = DataLoader(train_dataset, sampler=RandomSampler(train_dataset), batch_size=batch_size)
# val_dataloader = DataLoader(val_dataset, sampler=SequentialSampler(val_dataset), batch_size=batch_size)


In [20]:
# tokenizer = BertTokenizer.from_pretrained('distilbert-base-uncased')
# model = BertForSequenceClassification.from_pretrained('distilbert-base-uncased', num_labels=2).to('cpu')



The tokenizer class you load from this checkpoint is not the same type as the class this function is called from. It may result in unexpected tokenization. 
The tokenizer class you load from this checkpoint is 'DistilBertTokenizer'. 
The class this function is called from is 'BertTokenizer'.
You are using a model of type distilbert to instantiate a model of type bert. This is not supported for all configurations of models and can yield errors.
Some weights of the model checkpoint at distilbert-base-uncased were not used when initializing BertForSequenceClassification: ['distilbert.transformer.layer.3.sa_layer_norm.bias', 'distilbert.transformer.layer.1.ffn.lin1.bias', 'distilbert.transformer.layer.5.attention.q_lin.weight', 'distilbert.transformer.layer.0.attention.q_lin.weight', 'distilbert.transformer.layer.4.output_layer_norm.bias', 'distilbert.transformer.layer.5.ffn.lin2.bias', 'distilbert.transformer.layer.5.ffn.lin1.bias', 'vocab_transform.weight', 'distilbert.transformer.layer.

In [14]:
# import torch

# def get_gpu_memory_status(device=None):
#     if device is None:
#         device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
        
#     if device.type == 'cuda':
#         torch.cuda.empty_cache()
#         allocated_memory = torch.cuda.memory_allocated(device)
#         free_memory = torch.cuda.memory_reserved(device) - allocated_memory
#         return allocated_memory, free_memory
#     else:
#         return None, None

# if torch.cuda.is_available():
#     device = torch.device('cuda')
#     allocated_memory, free_memory = get_gpu_memory_status(device)
#     print(f"Allocated memory: {allocated_memory / 1024 ** 2} MB")
#     print(f"Free memory: {free_memory / 1024 ** 2} MB")
# else:
#     print("CUDA is not available on this system.")


Allocated memory: 0.0 MB
Free memory: 0.0 MB


In [32]:
# optimizer = AdamW(model.parameters(), lr=2e-5, eps=1e-8)
# epochs = 3
# scheduler = get_linear_schedule_with_warmup(optimizer, num_warmup_steps=0, num_training_steps=len(train_dataloader) * epochs)


In [33]:
# def train_model(model, dataloader, optimizer, scheduler, device):
#     model.train()
#     total_loss = 0

#     for batch in dataloader:
#         input_ids = batch['input_ids'].to(device)
#         attention_mask = batch['attention_mask'].to(device)
#         labels


In [35]:
# pip install tqdm


You should consider upgrading via the '/home/seankhatiri/sean2/venv/bin/python -m pip install --upgrade pip' command.
Note: you may need to restart the kernel to use updated packages.


In [ ]:
# from tqdm import tqdm
# import multiprocessing

# def train_model(model, dataloader, optimizer, scheduler, device):
#     model.train()
#     total_loss = 0

   
#     pool = multiprocessing.Pool()

    
#     progress_bar = tqdm(dataloader, desc="Training", position=0, leave=True)


#     results = []

#     for batch in progress_bar:
#         input_ids = batch['input_ids'].to(device)
#         attention_mask = batch['attention_mask'].to(device)
#         labels = batch['label'].to(device)

#         model.zero_grad()

#         result = pool.apply_async(model, (input_ids, attention_mask, labels))
#         results.append(result)

#     pool.close()
#     pool.join()

#     for result in tqdm(results, desc="Calculating loss", position=1, leave=True):
#         outputs = result.get()
#         loss = outputs[0]
#         total_loss += loss.item()

#     avg_train_loss = total_loss / len(dataloader)

#     return avg_train_loss

# # Evaluation loop
# def evaluate_model(model, dataloader, device):
#     model.eval()
#     total_loss = 0

#     # Adding a progress bar
#     progress_bar = tqdm(dataloader, desc="Evaluating", position=0, leave=True)

#     for batch in progress_bar:
#         input_ids = batch['input_ids'].to(device)
#         attention_mask = batch['attention_mask'].to(device)
#         labels = batch['label'].to(device)

#         with torch.no_grad():
#             outputs = model(input_ids, attention_mask=attention_mask, labels=labels)
#             loss = outputs[0]
#             total_loss += loss.item()

#     avg_val_loss = total_loss / len(dataloader)

#     return avg_val_loss


# for epoch in range(epochs):
#     train_loss = train_model(model, train_dataloader, optimizer, scheduler, 'cpu')
#     val_loss = evaluate_model(model, val_dataloader, 'cpu')

#     print(f"\nEpoch: {epoch + 1}")
#     print(f"Train Loss: {train_loss:.4f}")
#     print(f"Validation Loss: {val_loss:.4f}")

#     val_predictions, val_true_labels = get_predictions(model, val_dataloader, 'cuda')
#     val_predicted_labels = np.argmax(val_predictions, axis=1)

#     print("\nPerformance metrics on validation dataset:")
#     print("Accuracy:", accuracy_score(val_true_labels, val_predicted_labels))
#     print("\nClassification report:")
#     print(classification_report(val_true_labels, val_predicted_labels))

# val_predictions, val_true_labels = get_predictions(model, val_dataloader, 'cuda')
# val_predicted_labels = np.argmax(val_predictions, axis=1)

# print("\nPerformance metrics on validation dataset:")
# print("Accuracy:", accuracy_score(val_true_labels, val_predicted_labels))
# print("\nClassification report:")
# print(classification_report(val_true_labels, val_predicted_labels))



Training: 100%|█████████████████████████████████████████████████████████████████| 29489/29489 [14:12<00:00, 34.59it/s]


In [10]:
from sklearn.feature_extraction.text import CountVectorizer
cv = CountVectorizer(ngram_range=(1, 2))
X_train_cv = cv.fit_transform(X_train['Tokens'])

In [12]:
from sklearn.model_selection import cross_val_score
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import precision_score, recall_score, f1_score

# n_jobs=-1 to use all avaliable CPUs
lr = LogisticRegression(n_jobs=-1)

# Perform 10 fold cross validation on training data
cv_scores = cross_val_score(lr, X_train_cv, y_train, cv=10, n_jobs=-1)

print('Mean CV accuracy:', cv_scores.mean())

# Fitting the model
lr.fit(X_train_cv, y_train)

# Transform testing data using CountVectorizer
X_test_cv = cv.transform(X_test["Tokens"])

y_pred = lr.predict(X_test_cv)
accuracy = (y_pred == y_test).mean()
print('LogisticRegression Accuracy:', accuracy)

precision = precision_score(y_test, y_pred)
recall = recall_score(y_test, y_pred)
f1 = f1_score(y_test, y_pred)

print('Precision:', precision)
print('Recall:', recall)
print('F1 Score:', f1)



/home/seankhatiri/sean2/venv/lib/python3.9/site-packages/sklearn/linear_model/_logistic.py:458: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(
/home/seankhatiri/sean2/venv/lib/python3.9/site-packages/sklearn/linear_model/_logistic.py:458: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#l

Mean CV accuracy: 0.8945643848020517


/home/seankhatiri/sean2/venv/lib/python3.9/site-packages/sklearn/linear_model/_logistic.py:458: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(


LogisticRegression Accuracy: 0.8940603964257253
Precision: 0.9006877437897762
Recall: 0.9916556790355999
F1 Score: 0.9439852252963009


In [53]:
from sklearn.naive_bayes import MultinomialNB
from sklearn.metrics import accuracy_score

nb = MultinomialNB()

cv_scores = cross_val_score(nb, X_train_cv, y_train, cv=10)

print('Mean CV accuracy:', cv_scores.mean())

nb.fit(X_train_cv, y_train)
y_pred_nb = nb.predict(X_test_cv)
accuracy_nb = accuracy_score(y_test, y_pred_nb)
print('Multinomial Naive Bayes Accuracy:', accuracy_nb)

precision = precision_score(y_test, y_pred_nb)
recall = recall_score(y_test, y_pred_nb)
f1 = f1_score(y_test, y_pred_nb)

print('Precision:', precision)
print('Recall:', recall)
print('F1 Score:', f1)

Mean CV accuracy: 0.898036090850131
Multinomial Naive Bayes Accuracy: 0.9000288247961069
Precision: 0.9001797646101143
Recall: 0.9998116406102844
F1 Score: 0.9473834511315771


In [55]:
from sklearn.svm import LinearSVC

svm = LinearSVC()

cv_scores = cross_val_score(svm, X_train_cv, y_train, cv=10)

print('Mean CV accuracy:', cv_scores.mean())

svm.fit(X_train_cv, y_train)
y_pred_svm = svm.predict(X_test_cv)
accuracy_svm = accuracy_score(y_test, y_pred_svm)
print('Support Vector Machine Accuracy:', accuracy_svm)

precision = precision_score(y_test, y_pred_svm)
recall = recall_score(y_test, y_pred_svm)
f1 = f1_score(y_test, y_pred_svm)

print('Precision:', precision)
print('Recall:', recall)
print('F1 Score:', f1)

C:\Users\TRSM\anaconda3\lib\site-packages\sklearn\svm\_base.py:1244: ConvergenceWarning: Liblinear failed to converge, increase the number of iterations.
  warnings.warn(
C:\Users\TRSM\anaconda3\lib\site-packages\sklearn\svm\_base.py:1244: ConvergenceWarning: Liblinear failed to converge, increase the number of iterations.
  warnings.warn(
C:\Users\TRSM\anaconda3\lib\site-packages\sklearn\svm\_base.py:1244: ConvergenceWarning: Liblinear failed to converge, increase the number of iterations.
  warnings.warn(
C:\Users\TRSM\anaconda3\lib\site-packages\sklearn\svm\_base.py:1244: ConvergenceWarning: Liblinear failed to converge, increase the number of iterations.
  warnings.warn(
C:\Users\TRSM\anaconda3\lib\site-packages\sklearn\svm\_base.py:1244: ConvergenceWarning: Liblinear failed to converge, increase the number of iterations.
  warnings.warn(
C:\Users\TRSM\anaconda3\lib\site-packages\sklearn\svm\_base.py:1244: ConvergenceWarning: Liblinear failed to converge, increase the number of ite

Mean CV accuracy: 0.8569690672625404


C:\Users\TRSM\anaconda3\lib\site-packages\sklearn\svm\_base.py:1244: ConvergenceWarning: Liblinear failed to converge, increase the number of iterations.
  warnings.warn(


Support Vector Machine Accuracy: 0.8555708157417298
Precision: 0.9015784921435779
Recall: 0.9424373705029195
F1 Score: 0.9215552649512827


In [ ]:
from sklearn.ensemble import RandomForestClassifier, GradientBoostingClassifier
from sklearn.metrics import accuracy_score

rf = RandomForestClassifier(n_jobs=-1)  # Use all available CPU cores

cv_scores = cross_val_score(rf, X_train_cv, y_train, cv=10, n_jobs=-1)

print('Mean CV accuracy:', cv_scores.mean())

rf.fit(X_train_cv, y_train)

y_pred_rf = rf.predict(X_test_cv)
accuracy_rf = accuracy_score(y_test, y_pred_rf)
print('Random Forest Accuracy:', accuracy_rf)

precision = precision_score(y_test, y_pred_rf)
recall = recall_score(y_test, y_pred_rf)
f1 = f1_score(y_test, y_pred_rf)

print('Precision:', precision)
print('Recall:', recall)
print('F1 Score:', f1)


In [ ]:
from sklearn.ensemble import GradientBoostingClassifier

gb = GradientBoostingClassifier()
gb.fit(X_train_cv, y_train)

cv_scores = cross_val_score(gb, X_train_cv, y_train, cv=10)

print('Mean CV accuracy:', cv_scores.mean())

y_pred_gb = gb.predict(X_test_cv)
accuracy_gb = accuracy_score(y_test, y_pred_gb)
print('Gradient Boosting Accuracy:', accuracy_gb)

precision = precision_score(y_test, y_pred_gb)
recall = recall_score(y_test, y_pred_gb)
f1 = f1_score(y_test, y_pred_gb)

print('Precision:', precision)
print('Recall:', recall)
print('F1 Score:', f1)

In [15]:
import xgboost as xgb

y_train_mapped = (y_train + 1) // 2
y_test_mapped = (y_test + 1) // 2

xgb_clf = xgb.XGBClassifier()

cv_scores = cross_val_score(xgb_clf, X_train_cv, y_train_mapped, cv=10)

print('Mean CV accuracy:', cv_scores.mean())

xgb_clf.fit(X_train_cv, y_train_mapped)

y_pred_xgb = xgb_clf.predict(X_test_cv)
accuracy_xgb = accuracy_score(y_test_mapped, y_pred_xgb)
print('XGBoost Accuracy:', accuracy_xgb)

precision = precision_score(y_test_mapped, y_pred_xgb)
recall = recall_score(y_test_mapped, y_pred_xgb)
f1 = f1_score(y_test_mapped, y_pred_xgb)

print('Precision:', precision)
print('Recall:', recall)
print('F1 Score:', f1)

Mean CV accuracy: 0.896710011236013
XGBoost Accuracy: 0.8991331242376721
Precision: 0.8993094133281049
Recall: 0.9997820295471058
F1 Score: 0.946887938251924


In [7]:
pip install xgboost

Note: you may need to restart the kernel to use updated packages.


In [ ]:
from sklearn.tree import DecisionTreeClassifier
from sklearn.metrics import accuracy_score

dt = DecisionTreeClassifier()

cv_scores = cross_val_score(dt, X_train_cv, y_train, cv=10)

print('Mean CV accuracy:', cv_scores.mean())

dt.fit(X_train_cv, y_train)

y_pred_dt = dt.predict(X_test_cv)
accuracy_dt = accuracy_score(y_test, y_pred_dt)
print('Decision Tree Accuracy:', accuracy_dt)

precision = precision_score(y_test, y_pred_dt)
recall = recall_score(y_test, y_pred_dt)
f1 = f1_score(y_test, y_pred_dt)

print('Precision:', precision)
print('Recall:', recall)
print('F1 Score:', f1)

In [ ]:
from sklearn.ensemble import VotingClassifier
import joblib
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score
from sklearn.feature_extraction.text import CountVectorizer

models = joblib.load('models.pkl')

ensemble = VotingClassifier(estimators=[
    ('lr', models['lr']),
    ('nb', models['nb']),
    ('svm', models['svm'])],
    voting='hard')

cv = CountVectorizer()
X_test_cv = cv.fit_transform(X_test)

y_pred_ensemble = ensemble.predict(X_test_cv)
accuracy_ensemble = accuracy_score(y_test, y_pred_ensemble)
precision_ensemble = precision_score(y_test, y_pred_ensemble)
recall_ensemble = recall_score(y_test, y_pred_ensemble)
f1_ensemble = f1_score(y_test, y_pred_ensemble)

print('Ensemble Accuracy:', accuracy_ensemble)
print('Precision:', precision_ensemble)
print('Recall:', recall_ensemble)
print('F1 Score:', f1_ensemble)
